In [45]:
from utils.json_utils import *

In [46]:
original_data_path = "/home/guest-pjy/data/0831/hotpot_cf_cleaned.json"
new_data_path = "/home/guest-pjy/data/0831/hotpot_cf_cleaned.json"
data = load_json(original_data_path)
for idx, item in enumerate(data):
    item["index"] = idx
for item in data:
    if len(item["paraphrase"]) != 10:
        print(item["index"])
    if len(item["counterfactual"]) != 9:
        print(item["index"])
    for cf in item["counterfactual"]:
        if len(cf["contexts"]) != 3:
            print(item["index"])

JSONDecodeError: Expecting ',' delimiter: line 90837 column 9 (char 10857702)

In [ ]:
for item in data:
    for cf in item["counterfactual"]:
        answer = item["answers"]
        for ans in answer:
            if cf["answers"][0].lower() == ans.lower():
                print(item["index"])
                print(item["question"])
                print(cf["answers"][0])
                print(ans)

clarify
although
some
often
never
however
is not
are not
do not
does not
was not
were not
doesn't
although
though
directly
frequently
frequent
while
another
other
incorrectly
incorrect
yet
despite
common
commonly
essential for clarity
clearly
but
instead

step 1: original answer -> new answer
step 2: 단어 제거

In [ ]:
import re

for idx, item in enumerate(data):
    new_cf = []
    for cf in item["counterfactual"]:
        cf_answer = cf["answers"][0]
        for ans in item["answers"]:
            ans_lower = re.escape(
                ans.lower()
            )  # Escape special characters in the answer and make it lowercase
            pattern = re.compile(
                r"\b" + ans_lower + r"[\.,;:!\?]*\b", re.IGNORECASE
            )  # Compile regex pattern to ignore punctuation
            updated_contexts = []
            for text in cf["contexts"]:
                updated_text = pattern.sub(
                    cf_answer, text
                )  # Substitute matches in the original text
                updated_contexts.append(
                    updated_text
                )  # Collect the updated text
            cf["contexts"] = (
                updated_contexts  # Replace the contexts with the updated ones
            )
        new_cf.append(cf)
    data[idx][
        "counterfactual"
    ] = new_cf  # Ensure the updated counterfactuals are assigned back to data

In [ ]:
import re
from tqdm import tqdm

# List of words to remove
words_to_remove = [
    "clarify",
    "although",
    "some",
    "often",
    "never",
    "however",
    "although",
    "though",
    "directly",
    "frequently",
    "frequent",
    "while",
    "another",
    "other",
    "incorrectly",
    "incorrect",
    "yet",
    "despite",
    "common",
    "commonly",
    "essential for clarity",
    "clearly",
    "but",
    "instead",
    "nevertheless",
    "humor",
    "humorous",
    "joke",
    "joking",
    "misunderstandings",
    "misconceptions",
    "confuse",
    "confusing",
    "confusion",
    "confusions",
    "confused",
]

# Dictionary of negative phrases and their positive counterparts
negative_to_positive = {
    "is not": "is",
    "are not": "are",
    "was not": "was",
    "were not": "were",
    "has not": "has",
    "have not": "have",
    "had not": "had",
    "does not": "does",
    "do not": "do",
    "did not": "did",
    "isn't": "is",
    "aren't": "are",
    "wasn't": "was",
    "weren't": "were",
    "hasn't": "has",
    "haven't": "have",
    "hadn't": "had",
    "doesn't": "does",
    "don't": "do",
    "didn't": "did",
}

# Compile regex for removing words (case-insensitive)
remove_pattern = re.compile(
    r"\b(?:" + "|".join(map(re.escape, words_to_remove)) + r")\b[\.,;:!\?]*",
    re.IGNORECASE,
)

# Compile regex for removing "mis" from words starting with "mis", excluding "miss"
mis_pattern = re.compile(r"\bmis(?!s)\w*\b", re.IGNORECASE)


# Function to replace negative phrases
def replace_negatives(text, replacements):
    for neg, pos in replacements.items():
        # Adjust the regex to ignore punctuation after the negative phrase
        text = re.sub(
            r"\b" + re.escape(neg) + r"[\.,;:!\?]*\b",
            pos,
            text,
            flags=re.IGNORECASE,
        )
    return text


# Function to remove "mis" from words starting with "mis", excluding "miss"
def remove_mis(text):
    return mis_pattern.sub(
        lambda match: match.group().replace("mis", "", 1), text
    )


for idx, item in enumerate(tqdm(data)):
    new_cf = []
    for cf in item["counterfactual"]:
        cf_answer = cf["answers"][0]
        for ans in item["answers"]:
            pattern = re.compile(re.escape(ans), re.IGNORECASE)
            for i, text in enumerate(cf["contexts"]):
                # Remove unwanted words and replace negative phrases
                cleaned_text = remove_pattern.sub("", text)
                cleaned_text = replace_negatives(
                    cleaned_text, negative_to_positive
                )

                # Remove "mis" from words starting with "mis", excluding "miss"
                cleaned_text = remove_mis(cleaned_text)

                # Replace the answer in the cleaned text
                updated_text = pattern.sub(cf_answer, cleaned_text)

                # Update the context with the replaced and cleaned text
                cf["contexts"][
                    i
                ] = (
                    updated_text.strip()
                )  # strip() removes leading/trailing spaces
        new_cf.append(cf)
    data[idx]["counterfactual"] = new_cf

100%|██████████| 3000/3000 [00:14<00:00, 207.74it/s]


In [ ]:
save_json(new_data_path, data)

In [ ]:
from utils.json_utils import *

original_data_path = "/home/guest-pjy/data/0831/hotpot_cf_cleaned.json"
data = load_json(original_data_path)
for idx, item in enumerate(data):
    item["index"] = idx
for item in data:
    if len(item["paraphrase"]) != 10:
        print(item["index"])
    if len(item["counterfactual"]) != 9:
        print(item["index"])
    for cf in item["counterfactual"]:
        if len(cf["contexts"]) != 3:
            print(item["index"])

In [ ]:
len(data)

3000